<a href="https://colab.research.google.com/github/pandey-rakshit/mavenflix-analytics/blob/main/notebooks/mavenflix_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MavenFlix Subscription Analysis

## Executive Summary: MavenFlix Subscription Analysis

This analysis of MavenFlix subscription data from September 2022 to September 2023 reveals critical insights into subscriber behavior, longevity, and retention.

**Key Findings:**

1.  **High Early Churn:** A significant 65.30% of subscriptions have been canceled, with 32.09% of these cancellations occurring within the first month. The average subscription duration is short at 77.2 days, with 854 customers churning within the first month.

2.  **Payment Status Impact:** Unpaid subscriptions exhibit an alarming churn rate of 92.48%, drastically higher than paid subscriptions (64.07%). This highlights payment issues as a primary driver of churn.

3.  **Fluctuating Monthly Trends:** While new subscriptions peaked at 306 in July 2023, cancellations also saw highs, reaching 225 in June 2023. Despite this, cumulative net subscriber growth shows an upward trend, reaching 1065 active subscribers by September 2023.

4.  **Long-Term Retention Challenge:** Only 16.16% of customers maintain subscriptions for 5 or more months. Older cohorts (e.g., September 2022) show very low long-term retention (9.22%), while recent cohorts demonstrate higher initial retention, which is expected due to their recency.

5.  **Month-over-Month Stability:** The average month-over-month retention rate is 78.64%, with an average churn rate of 21.36%, indicating a relatively stable but persistent monthly churn.

**Conclusion:**

MavenFlix faces a substantial challenge with early subscriber churn, particularly among unpaid customers. While the platform shows consistent new subscriber acquisition, converting these into long-term, retained customers is critical. Addressing the root causes of early churn, especially for unpaid subscriptions, and implementing strategies to improve long-term retention across cohorts will be essential for enhancing subscriber lifetime value and ensuring sustainable growth.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
data_path = "https://raw.githubusercontent.com/pandey-rakshit/mavenflix-analytics/refs/heads/main/data/DATA_ANALYSIS_ALMABETTER.csv"

In [3]:
# load data - update path as needed
df = pd.read_csv(data_path)
print(f"Shape: {df.shape}")
df.head()

Shape: (3069, 6)


,customer_id,created_date,canceled_date,subscription_cost,subscription_interval,was_subscription_paid
0,154536156,2022-09-01,NaN,39,month,Yes
1,149713408,2022-09-01,2022-09-02,39,month,No
2,153756284,2022-09-01,2022-09-02,39,month,No
3,121253113,2022-09-01,2022-09-23,39,month,Yes
4,154467210,2022-09-01,2023-06-29,39,month,Yes


## Data Understanding

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3069 entries, 0 to 3068
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            3069 non-null   int64 
 1   created_date           3069 non-null   object
 2   canceled_date          2004 non-null   object
 3   subscription_cost      3069 non-null   int64 
 4   subscription_interval  3069 non-null   object
 5   was_subscription_paid  3069 non-null   object
dtypes: int64(2), object(4)
memory usage: 144.0+ KB


In [5]:
# check missing values
df.isnull().sum()

customer_id                 0
created_date                0
canceled_date            1065
subscription_cost           0
subscription_interval       0
was_subscription_paid       0
dtype: int64

In [6]:
# check unique values per column
df.nunique()

customer_id              2877
created_date              370
canceled_date             360
subscription_cost           1
subscription_interval       1
was_subscription_paid       2
dtype: int64

In [7]:
# duplicate rows check
print(f"Duplicate rows: {df.duplicated().sum()}")

Duplicate rows: 0


## Data Preparation

In [8]:
# convert date columns
df['created_date'] = pd.to_datetime(df['created_date'])
df['canceled_date'] = pd.to_datetime(df['canceled_date'])

In [9]:
# check date range
print(f"Created date range: {df['created_date'].min().date()} to {df['created_date'].max().date()}")
print(f"Canceled date range: {df['canceled_date'].min().date()} to {df['canceled_date'].max().date()}")

Created date range: 2022-09-01 to 2023-09-08
Canceled date range: 2022-09-02 to 2023-09-08


In [10]:
# check for invalid dates (canceled before created)
invalid = df[df['canceled_date'] < df['created_date']]
print(f"Invalid date rows: {len(invalid)}")

Invalid date rows: 0


In [11]:
# reference date for active customer duration calculation
reference_date = pd.to_datetime('2023-09-08')

# create derived columns
df['is_active'] = df['canceled_date'].isnull().astype(int)

df['duration_days'] = np.where(
    df['is_active'] == 1,
    (reference_date - df['created_date']).dt.days,
    (df['canceled_date'] - df['created_date']).dt.days
)

df['duration_months'] = (df['duration_days'] / 30).round(1)

df['cohort_month'] = df['created_date'].dt.to_period('M')
df['cancel_month'] = df['canceled_date'].dt.to_period('M')
df['created_ym'] = df['created_date'].dt.strftime('%Y-%m')

df['is_paid'] = (df['was_subscription_paid'] == 'Yes').astype(int)

In [12]:
# verify derived columns
df[['customer_id', 'created_date', 'canceled_date', 'is_active', 'duration_days', 'duration_months', 'cohort_month']].head(10)

,customer_id,created_date,canceled_date,is_active,duration_days,duration_months,cohort_month
0,154536156,2022-09-01,NaT,1,372.0,12.4,2022-09
1,149713408,2022-09-01,2022-09-02,0,1.0,0.0,2022-09
2,153756284,2022-09-01,2022-09-02,0,1.0,0.0,2022-09
3,121253113,2022-09-01,2022-09-23,0,22.0,0.7,2022-09
4,154467210,2022-09-01,2023-06-29,0,301.0,10.0,2022-09
5,154510811,2022-09-01,2023-05-08,0,249.0,8.3,2022-09
6,154494346,2022-09-01,2022-11-26,0,86.0,2.9,2022-09
7,148671505,2022-09-01,2022-11-23,0,83.0,2.8,2022-09
8,154568379,2022-09-01,2022-10-25,0,54.0,1.8,2022-09
9,154428434,2022-09-01,2022-10-31,0,60.0,2.0,2022-09


## Basic Statistics

In [13]:
# active vs canceled
active_counts = df['is_active'].value_counts()
print(f"Canceled: {active_counts[0]} ({active_counts[0]/len(df)*100:.2f}%)")
print(f"Active: {active_counts[1]} ({active_counts[1]/len(df)*100:.2f}%)")

Canceled: 2004 (65.30%)
Active: 1065 (34.70%)


In [14]:
# payment status
df['was_subscription_paid'].value_counts()

was_subscription_paid
Yes    2936
No      133
Name: count, dtype: int64

In [15]:
# duration statistics
df['duration_days'].describe()

count    3069.000000
mean       77.202020
std        76.067507
min         0.000000
25%        26.000000
50%        52.000000
75%       107.000000
max       372.000000
Name: duration_days, dtype: float64

In [16]:
# returning customers (multiple subscriptions)
total_rows = len(df)
unique_customers = df['customer_id'].nunique()
repeat_subs = total_rows - unique_customers

print(f"Total records: {total_rows}")
print(f"Unique customers: {unique_customers}")
print(f"Repeat subscriptions: {repeat_subs} ({repeat_subs/total_rows*100:.2f}%)")

Total records: 3069
Unique customers: 2877
Repeat subscriptions: 192 (6.26%)


## Subscription Trends

In [17]:
# monthly new subscriptions
monthly_new = df.groupby('created_ym').size().reset_index(name='new_subs')
monthly_new

,created_ym,new_subs
0,2022-09,217
1,2022-10,196
2,2022-11,189
3,2022-12,230
4,2023-01,273
5,2023-02,275
6,2023-03,266
7,2023-04,223
8,2023-05,285
9,2023-06,275


In [18]:
# monthly cancellations
canceled_df = df[df['is_active'] == 0].copy()
canceled_df['cancel_ym'] = canceled_df['canceled_date'].dt.strftime('%Y-%m')
monthly_cancel = canceled_df.groupby('cancel_ym').size().reset_index(name='cancellations')
monthly_cancel

,cancel_ym,cancellations
0,2022-09,33
1,2022-10,81
2,2022-11,109
3,2022-12,123
4,2023-01,149
5,2023-02,171
6,2023-03,224
7,2023-04,194
8,2023-05,215
9,2023-06,225


In [19]:
# merge trends
trends = monthly_new.merge(monthly_cancel, left_on='created_ym', right_on='cancel_ym', how='left')
trends = trends[['created_ym', 'new_subs', 'cancellations']].fillna(0)
trends['cancellations'] = trends['cancellations'].astype(int)
trends['net_change'] = trends['new_subs'] - trends['cancellations']
trends['cumulative'] = trends['net_change'].cumsum()
trends.columns = ['month', 'new_subs', 'cancellations', 'net_change', 'cumulative']
trends

,month,new_subs,cancellations,net_change,cumulative
0,2022-09,217,33,184,184
1,2022-10,196,81,115,299
2,2022-11,189,109,80,379
3,2022-12,230,123,107,486
4,2023-01,273,149,124,610
5,2023-02,275,171,104,714
6,2023-03,266,224,42,756
7,2023-04,223,194,29,785
8,2023-05,285,215,70,855
9,2023-06,275,225,50,905


In [20]:
# peak and low months
print(f"Highest acquisition: {trends.loc[trends['new_subs'].idxmax(), 'month']} ({trends['new_subs'].max()} subs)")
print(f"Lowest acquisition: {trends.loc[trends['new_subs'].idxmin(), 'month']} ({trends['new_subs'].min()} subs)")
print(f"Highest cancellations: {trends.loc[trends['cancellations'].idxmax(), 'month']} ({trends['cancellations'].max()} cancels)")

Highest acquisition: 2023-07 (306 subs)
Lowest acquisition: 2023-09 (66 subs)
Highest cancellations: 2023-06 (225 cancels)


## Customer Longevity

In [21]:
# customers with 5+ months subscription
customers_5plus = (df['duration_months'] >= 5).sum()
pct_5plus = customers_5plus / len(df) * 100

print(f"Customers with 5+ months: {customers_5plus}")
print(f"Percentage: {pct_5plus:.2f}%")

Customers with 5+ months: 496
Percentage: 16.16%


In [22]:
# early churn (within first month)
early_churn = ((df['is_active'] == 0) & (df['duration_months'] < 1)).sum()
total_canceled = (df['is_active'] == 0).sum()

print(f"Churned within first month: {early_churn}")
print(f"Percentage of all cancellations: {early_churn/total_canceled*100:.2f}%")

Churned within first month: 643
Percentage of all cancellations: 32.09%


In [23]:
# duration buckets
bins = [0, 1, 2, 3, 4, 5, 6, 9, 12, float('inf')]
labels = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-9', '9-12', '12+']
df['duration_bucket'] = pd.cut(df['duration_months'], bins=bins, labels=labels, right=False)
df['duration_bucket'].value_counts().sort_index()

duration_bucket
0-1     854
1-2     794
2-3     461
3-4     275
4-5     189
5-6     150
6-9     239
9-12     96
12+      11
Name: count, dtype: int64

## Payment Analysis

In [24]:
# churn by payment status
payment_analysis = df.groupby('was_subscription_paid').agg(
    total=('customer_id', 'count'),
    active=('is_active', 'sum'),
    avg_duration=('duration_days', 'mean')
).round(2)

payment_analysis['churned'] = payment_analysis['total'] - payment_analysis['active']
payment_analysis['churn_rate'] = (payment_analysis['churned'] / payment_analysis['total'] * 100).round(2)
payment_analysis

,total,active,avg_duration,churned,churn_rate
was_subscription_paid,,,,,
No,133,10,18.73,123,92.48
Yes,2936,1055,79.85,1881,64.07


## Retention Analysis

In [25]:
# overall retention
retention_rate = df['is_active'].sum() / len(df) * 100
churn_rate = 100 - retention_rate

print(f"Overall Retention Rate: {retention_rate:.2f}%")
print(f"Overall Churn Rate: {churn_rate:.2f}%")

Overall Retention Rate: 34.70%
Overall Churn Rate: 65.30%


In [26]:
# retention by cohort
cohort_summary = df.groupby('cohort_month').agg(
    total=('customer_id', 'count'),
    active=('is_active', 'sum')
).reset_index()

cohort_summary['churned'] = cohort_summary['total'] - cohort_summary['active']
cohort_summary['retention_rate'] = (cohort_summary['active'] / cohort_summary['total'] * 100).round(2)
cohort_summary

,cohort_month,total,active,churned,retention_rate
0,2022-09,217,20,197,9.22
1,2022-10,196,28,168,14.29
2,2022-11,189,22,167,11.64
3,2022-12,230,43,187,18.70
4,2023-01,273,56,217,20.51
5,2023-02,275,58,217,21.09
6,2023-03,266,67,199,25.19
7,2023-04,223,77,146,34.53
8,2023-05,285,104,181,36.49
9,2023-06,275,136,139,49.45


In [27]:
# best and worst cohorts
best = cohort_summary.loc[cohort_summary['retention_rate'].idxmax()]
worst = cohort_summary.loc[cohort_summary['retention_rate'].idxmin()]

print(f"Best retention: {best['cohort_month']} ({best['retention_rate']}%)")
print(f"Worst retention: {worst['cohort_month']} ({worst['retention_rate']}%)")

Best retention: 2023-09 (96.97%)
Worst retention: 2022-09 (9.22%)


## Cohort Analysis

In [28]:
# build cohort retention table
cohort_data = []
cohort_months = sorted(df['cohort_month'].unique())

for cohort in cohort_months:
    cohort_df = df[df['cohort_month'] == cohort]
    cohort_size = len(cohort_df)

    for offset in range(13):
        target_month = cohort + offset

        still_active = cohort_df[
            (cohort_df['canceled_date'].isnull()) |
            (cohort_df['cancel_month'] > target_month)
        ].shape[0]

        retention = (still_active / cohort_size * 100) if cohort_size > 0 else 0

        cohort_data.append({
            'cohort': str(cohort),
            'month_offset': offset,
            'cohort_size': cohort_size,
            'retained': still_active,
            'retention_rate': round(retention, 2)
        })

cohort_df = pd.DataFrame(cohort_data)

In [29]:
# pivot for heatmap
cohort_pivot = cohort_df.pivot(index='cohort', columns='month_offset', values='retention_rate')
cohort_pivot

month_offset,0,1,2,3,4,5,6,7,8,9,10,11,12
cohort,,,,,,,,,,,,,
2022-09,84.79,58.53,43.78,34.56,29.49,25.35,20.74,17.05,13.36,11.06,11.06,9.22,9.22
2022-10,87.76,62.76,48.47,37.76,31.63,26.02,22.96,20.41,16.84,14.80,14.29,14.29,14.29
2022-11,85.19,62.96,46.03,33.33,23.28,19.05,17.99,16.93,15.87,11.64,11.64,11.64,11.64
2022-12,85.65,64.35,47.39,36.52,30.43,26.52,23.91,20.00,18.70,18.70,18.70,18.70,18.70
2023-01,86.81,67.77,50.18,39.93,32.60,27.47,23.81,21.25,20.51,20.51,20.51,20.51,20.51
2023-02,87.27,62.91,47.64,37.09,28.36,25.45,21.82,21.09,21.09,21.09,21.09,21.09,21.09
2023-03,83.46,62.03,45.11,34.59,30.08,25.56,25.19,25.19,25.19,25.19,25.19,25.19,25.19
2023-04,86.10,60.09,45.74,38.57,35.43,34.53,34.53,34.53,34.53,34.53,34.53,34.53,34.53
2023-05,86.32,62.81,48.77,39.30,36.49,36.49,36.49,36.49,36.49,36.49,36.49,36.49,36.49


## Key Metrics Summary

In [30]:
summary = {
    'Total Records': len(df),
    'Unique Customers': unique_customers,
    'Active Subscriptions': df['is_active'].sum(),
    'Canceled Subscriptions': (df['is_active'] == 0).sum(),
    'Retention Rate (%)': round(retention_rate, 2),
    'Churn Rate (%)': round(churn_rate, 2),
    'Paid Subscriptions': df['is_paid'].sum(),
    'Unpaid Subscriptions': (df['is_paid'] == 0).sum(),
    'Avg Duration (days)': round(df['duration_days'].mean(), 1),
    'Median Duration (days)': round(df['duration_days'].median(), 1),
    'Customers 5+ Months': customers_5plus,
    '5+ Months (%)': round(pct_5plus, 2)
}

pd.DataFrame(list(summary.items()), columns=['Metric', 'Value'])

,Metric,Value
0,Total Records,3069.00
1,Unique Customers,2877.00
2,Active Subscriptions,1065.00
3,Canceled Subscriptions,2004.00
4,Retention Rate (%),34.70
5,Churn Rate (%),65.30
6,Paid Subscriptions,2936.00
7,Unpaid Subscriptions,133.00
8,Avg Duration (days),77.20
9,Median Duration (days),52.00


---
## Visualizations

In [31]:
# 1. subscription status distribution
status_df = df['is_active'].value_counts().reset_index()
status_df['status'] = status_df['is_active'].map({0: 'Canceled', 1: 'Active'})

fig = px.pie(status_df, values='count', names='status',
             title='Subscription Status Distribution',
             color='status', color_discrete_map={'Canceled': '#EF553B', 'Active': '#00CC96'})
fig.show()

In [32]:
# 2. monthly new subscriptions
fig = px.bar(trends, x='month', y='new_subs',
             title='Monthly New Subscriptions',
             labels={'new_subs': 'New Subscriptions', 'month': 'Month'},
             text='new_subs')
fig.update_traces(textposition='outside')
fig.show()

In [33]:
# 3. monthly cancellations
fig = px.bar(trends, x='month', y='cancellations',
             title='Monthly Cancellations',
             labels={'cancellations': 'Cancellations', 'month': 'Month'},
             text='cancellations',
             color_discrete_sequence=['#EF553B'])
fig.update_traces(textposition='outside')
fig.show()

In [34]:
# 4. new vs canceled comparison
fig = go.Figure()
fig.add_trace(go.Bar(x=trends['month'], y=trends['new_subs'], name='New Subscriptions', marker_color='#636EFA'))
fig.add_trace(go.Bar(x=trends['month'], y=trends['cancellations'], name='Cancellations', marker_color='#EF553B'))
fig.update_layout(title='New Subscriptions vs Cancellations', barmode='group',
                  xaxis_title='Month', yaxis_title='Count')
fig.update_traces(textposition='outside')
fig.show()

In [35]:
# 5. net change trend

# line chart for retention rate
fig = go.Figure()

# retention line
fig.add_trace(go.Scatter(
    x=trends['month'],
    y=trends['net_change'],
    mode='lines+markers+text',
    name='Monthly Net Subscriber Change',
    line=dict(color='#2ecc71', width=3),
    marker=dict(size=10),
    text=trends['net_change'],
    textposition='top center'
))


fig.add_hline(y=0, line_dash='dash', line_color='red')


fig.update_layout(
    title='Monthly Net Subscriber Change',
    xaxis_title='Month',
    yaxis_title='Net Change',
    yaxis=dict(range=[0, 100]),
    height=500,
    template='plotly_white'
)


fig.show()

In [36]:
# 6. cumulative growth
fig = px.area(trends, x='month', y='cumulative',
              title='Cumulative Net Subscriber Growth',
              labels={'cumulative': 'Cumulative Net', 'month': 'Month'})
fig.show()

In [37]:
# 7. subscription duration distribution
fig = px.histogram(df, x='duration_days', nbins=30,
                   title='Subscription Duration Distribution',
                   labels={'duration_days': 'Duration (Days)', 'count': 'Frequency'})
fig.add_vline(x=df['duration_days'].mean(), line_dash='dash', line_color='red',
              annotation_text=f"Mean: {df['duration_days'].mean():.0f}")
fig.show()

In [38]:
# 8. duration by status
df['status_label'] = df['is_active'].map({0: 'Canceled', 1: 'Active'})

fig = px.box(df, x='status_label', y='duration_days',
             title='Subscription Duration by Status',
             labels={'status_label': 'Status', 'duration_days': 'Duration (Days)'},
             color='status_label', color_discrete_map={'Canceled': '#EF553B', 'Active': '#00CC96'})
fig.show()

In [39]:
# 9. payment status vs retention
payment_status = df.groupby(['was_subscription_paid', 'status_label']).size().reset_index(name='count')

fig = px.bar(payment_status, x='was_subscription_paid', y='count', color='status_label',
             title='Subscription Status by Payment',
             labels={'was_subscription_paid': 'Payment Status', 'count': 'Count'},
             color_discrete_map={'Canceled': '#EF553B', 'Active': '#00CC96'},
             text='count')
fig.update_traces(textposition='inside')
fig.show()

In [40]:
# list of months
months = pd.date_range('2022-09-01', '2023-09-01', freq='MS')

retention_data = []

for i in range(len(months) - 1):
    t_end = months[i] + pd.offsets.MonthEnd(0)
    t1_end = months[i+1] + pd.offsets.MonthEnd(0)

    # active at end of month t
    active_t = set(df[
        (df['created_date'] <= t_end) &
        ((df['canceled_date'].isna()) | (df['canceled_date'] > t_end))
    ]['customer_id'])

    # active at end of month t+1
    active_t1 = set(df[
        (df['created_date'] <= t1_end) &
        ((df['canceled_date'].isna()) | (df['canceled_date'] > t1_end))
    ]['customer_id'])

    # retained
    retained = active_t & active_t1
    churned = len(active_t) - len(retained)

    retention_rate = (len(retained) / len(active_t) * 100) if len(active_t) > 0 else 0
    churn_rate = 100 - retention_rate

    retention_data.append({
        'month': months[i].strftime('%Y-%m'),
        'active_at_month_end': len(active_t),
        'retained_next_month': len(retained),
        'churned': churned,
        'retention_rate': round(retention_rate, 2),
        'churn_rate': round(churn_rate, 2)
    })

retention_df = pd.DataFrame(retention_data)

# display results
print("Month-over-Month Retention")
print(retention_df)

# best and worst months
best = retention_df.loc[retention_df['retention_rate'].idxmax()]
worst = retention_df.loc[retention_df['retention_rate'].idxmin()]
avg_retention = retention_df['retention_rate'].mean()

print(f"\nAverage Retention Rate: {avg_retention:.2f}%")
print(f"Best Month: {best['month']} ({best['retention_rate']}%)")
print(f"Worst Month: {worst['month']} ({worst['retention_rate']}%)")

Month-over-Month Retention
      month  active_at_month_end  retained_next_month  churned  \
0   2022-09                  184                  127       57   
1   2022-10                  299                  218       81   
2   2022-11                  379                  289       90   
3   2022-12                  486                  373      113   
4   2023-01                  610                  476      134   
5   2023-02                  714                  535      179   
6   2023-03                  756                  594      162   
7   2023-04                  785                  612      173   
8   2023-05                  855                  674      181   
9   2023-06                  905                  749      156   
10  2023-07                 1004                  829      175   
11  2023-08                 1053                 1001       52   

    retention_rate  churn_rate  
0            69.02       30.98  
1            72.91       27.09  
2            76

In [41]:
# line chart for retention rate
fig = go.Figure()

# retention line
fig.add_trace(go.Scatter(
    x=retention_df['month'],
    y=retention_df['retention_rate'],
    mode='lines+markers+text',
    name='Retention Rate',
    line=dict(color='#2ecc71', width=3),
    marker=dict(size=10),
    text=retention_df['retention_rate'],
    textposition='top center'
))

# average line
fig.add_hline(
    y=avg_retention,
    line_dash='dash',
    line_color='gray',
    annotation_text=f'Avg: {avg_retention:.1f}%',
    annotation_position='right'
)

fig.update_layout(
    title='Month-over-Month Retention Rate',
    xaxis_title='Month',
    yaxis_title='Retention Rate (%)',
    yaxis=dict(range=[0, 100]),
    height=500,
    template='plotly_white'
)

fig.show()

In [42]:

# list of months
months = pd.date_range('2022-09-01', '2023-09-01', freq='MS')

churn_data = []

for i in range(len(months) - 1):
    t_end = months[i] + pd.offsets.MonthEnd(0)
    t1_end = months[i+1] + pd.offsets.MonthEnd(0)

    # active at end of month t
    active_t = set(df[
        (df['created_date'] <= t_end) &
        ((df['canceled_date'].isna()) | (df['canceled_date'] > t_end))
    ]['customer_id'])

    # active at end of month t+1
    active_t1 = set(df[
        (df['created_date'] <= t1_end) &
        ((df['canceled_date'].isna()) | (df['canceled_date'] > t1_end))
    ]['customer_id'])

    # churned = active in t but not in t+1
    churned = active_t - active_t1

    churn_rate = (len(churned) / len(active_t) * 100) if len(active_t) > 0 else 0

    churn_data.append({
        'month': months[i].strftime('%Y-%m'),
        'active_at_month_end': len(active_t),
        'churned_next_month': len(churned),
        'churn_rate': round(churn_rate, 2)
    })

churn_df = pd.DataFrame(churn_data)

print("Month-over-Month Churn Rate")
print(churn_df)

# best and worst
best = churn_df.loc[churn_df['churn_rate'].idxmin()]  # lowest churn is best
worst = churn_df.loc[churn_df['churn_rate'].idxmax()]  # highest churn is worst
avg_churn = churn_df['churn_rate'].mean()

print(f"\nAverage Churn Rate: {avg_churn:.2f}%")
print(f"Best Month (lowest churn): {best['month']} ({best['churn_rate']}%)")
print(f"Worst Month (highest churn): {worst['month']} ({worst['churn_rate']}%)")

# plot
fig = px.line(
    churn_df,
    x='month',
    y='churn_rate',
    markers=True,
    text='churn_rate',
    title='Month-over-Month Churn Rate'
)

fig.update_traces(
    line=dict(color='#e74c3c', width=3),
    marker=dict(size=10),
    textposition='top center'
)

fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Churn Rate (%)',
    height=500,
    template='plotly_white'
)

fig.show()

Month-over-Month Churn Rate
      month  active_at_month_end  churned_next_month  churn_rate
0   2022-09                  184                  57       30.98
1   2022-10                  299                  81       27.09
2   2022-11                  379                  90       23.75
3   2022-12                  486                 113       23.25
4   2023-01                  610                 134       21.97
5   2023-02                  714                 179       25.07
6   2023-03                  756                 162       21.43
7   2023-04                  785                 173       22.04
8   2023-05                  855                 181       21.17
9   2023-06                  905                 156       17.24
10  2023-07                 1004                 175       17.43
11  2023-08                 1053                  52        4.94

Average Churn Rate: 21.36%
Best Month (lowest churn): 2023-08 (4.94%)
Worst Month (highest churn): 2022-09 (30.98%)


In [43]:
# dual bar chart
fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=retention_df['month'],
    y=retention_df['retained_next_month'],
    name='Retained',
    marker_color='#2ecc71'
))

fig2.add_trace(go.Bar(
    x=retention_df['month'],
    y=retention_df['churned'],
    name='Churned',
    marker_color='#e74c3c'
))

fig2.update_layout(
    title='Monthly Retained vs Churned Customers',
    xaxis_title='Month',
    yaxis_title='Customers',
    barmode='stack',
    height=500,
    template='plotly_white'
)

fig2.show()

In [44]:
# 11. customer longevity distribution
longevity = df['duration_bucket'].value_counts().sort_index().reset_index()
longevity.columns = ['bucket', 'count']

fig = px.bar(longevity, x='bucket', y='count',
             title='Customer Longevity Distribution',
             labels={'bucket': 'Duration (Months)', 'count': 'Customers'},
             text='count')
fig.update_traces(textposition='outside')
fig.show()

In [45]:
# get unique cohorts
cohorts = sorted(df['cohort_month'].unique())

# find last month in data
last_month = df['canceled_date'].max()
if pd.isna(last_month):
    last_month = df['created_date'].max()
last_period = last_month.to_period('M')

# build cohort MoM retention data
cohort_data = []

for cohort in cohorts:
    cohort_customers = df[df['cohort_month'] == cohort]['customer_id'].unique()
    cohort_size = len(cohort_customers)
    cohort_df = df[df['customer_id'].isin(cohort_customers)]
    cohort_start = cohort.to_timestamp()

    for offset in range(12):
        target_month = cohort + offset
        next_month = cohort + offset + 1

        # skip if beyond data range
        if target_month > last_period or next_month > last_period:
            continue

        t_end = target_month.to_timestamp() + pd.offsets.MonthEnd(0)
        t1_end = next_month.to_timestamp() + pd.offsets.MonthEnd(0)

        # active at end of month t
        active_t = set(cohort_df[
            (cohort_df['created_date'] <= t_end) &
            ((cohort_df['canceled_date'].isna()) | (cohort_df['canceled_date'] > t_end))
        ]['customer_id'])

        # active at end of month t+1
        active_t1 = set(cohort_df[
            (cohort_df['created_date'] <= t1_end) &
            ((cohort_df['canceled_date'].isna()) | (cohort_df['canceled_date'] > t1_end))
        ]['customer_id'])

        # retained = active in both
        retained = active_t & active_t1

        retention_rate = (len(retained) / len(active_t) * 100) if len(active_t) > 0 else None

        cohort_data.append({
            'cohort': str(cohort),
            'month_offset': offset,
            'retention_rate': round(retention_rate, 2) if retention_rate else None
        })

cohort_retention_df = pd.DataFrame(cohort_data)

# pivot for heatmap
cohort_pivot = cohort_retention_df.pivot(index='cohort', columns='month_offset', values='retention_rate')

print("Cohort MoM Retention Table")
print(cohort_pivot)

# heatmap
fig = px.imshow(
    cohort_pivot,
    text_auto='.1f',
    color_continuous_scale='RdYlGn',
    aspect='auto',
    labels=dict(x='Month Offset', y='Cohort', color='Retention %')
)

fig.update_layout(
    title='Cohort Month-over-Month Retention Heatmap',
    height=600,
    width=1000
)

fig.show()

Cohort MoM Retention Table
month_offset     0      1      2      3      4      5      6      7      8   \
cohort                                                                        
2022-09       69.02  74.80  78.95  85.33  85.94  81.82  82.22  82.05  85.29   
2022-10       71.51  77.24  77.89  85.53  84.85  87.50  89.80  84.78  90.00   
2022-11       73.91  73.11  73.86  69.23  82.61  94.87  95.00  95.00  80.00   
2022-12       75.13  73.65  77.06  83.53  87.50  90.77  85.00  94.44  96.36   
2023-01       78.06  74.19  79.86  81.98  84.78  86.25  89.86  96.92    NaN   
2023-02       72.20  75.43  78.03  77.14  89.16  84.62  97.18    NaN    NaN   
2023-03       74.32  73.49  77.87  88.66  86.81  97.67    NaN    NaN    NaN   
2023-04       70.31  77.78  85.98  91.40  96.63    NaN    NaN    NaN    NaN   
2023-05       73.17  78.57  80.95  91.94    NaN    NaN    NaN    NaN    NaN   
2023-06       74.89  82.39  95.39    NaN    NaN    NaN    NaN    NaN    NaN   
2023-07       74.33  91.4

## Overall Notebook Summary: MavenFlix Subscription Analysis

This analysis explores MavenFlix subscription data from September 2022 to September 2023 to understand subscription trends, customer longevity, and retention patterns.

### 1. Data Overview & Preparation:
- The dataset contains **3069 records** and **6 columns**, with no duplicate rows.
- Dates were converted to datetime objects; `canceled_date` had **1065 missing values**, indicating active subscriptions.
- Derived columns like `is_active`, `duration_days`, `duration_months`, `cohort_month`, `cancel_month`, and `is_paid` were created for further analysis.

### 2. Basic Statistics:
- **65.30% (2004)** of subscriptions have been canceled, while **34.70% (1065)** are still active.
- The vast majority of subscriptions (**2936 out of 3069**) were paid, with only 133 marked as unpaid.
- The average subscription duration was **77.2 days**, with a median of **52 days**, and a maximum of **372 days**.
- **192 (6.26%)** repeat subscriptions were identified, suggesting a small but present segment of returning customers.

### 3. Subscription Trends:
- Monthly new subscriptions fluctuated, with a high of **306 in July 2023** and a low of **66 in September 2023**.
- Monthly cancellations also varied, peaking at **225 in June 2023**.
- The cumulative net subscriber growth showed an upward trend, reaching **1065 active subscribers by September 2023**.

### 4. Customer Longevity:
- **496 customers (16.16%)** maintained their subscription for **5 or more months**.
- A significant portion of cancellations (**32.09%**) occurred within the **first month**, highlighting early churn as a critical area.
- The `0-1` month bucket had the highest number of customers (**854**), followed by `1-2` months (**794**), indicating a high drop-off in the early stages.

### 5. Payment Analysis:
- **Unpaid subscriptions had a significantly higher churn rate (92.48%)** compared to paid subscriptions (64.07%). This indicates payment issues are a strong predictor of churn.
- Active unpaid subscriptions had a much lower average duration (18.73 days) than active paid subscriptions (79.85 days).

### 6. Retention Analysis:
- The **overall retention rate was 34.70%**, with a corresponding churn rate of 65.30%.
- **Cohort retention rates varied significantly**: The **September 2023 cohort** showed the best retention (96.97%, likely due to its recency), while the **September 2022 cohort** had the worst (9.22%) over the full period.

### 7. Month-over-Month Retention & Churn:
- **Average Month-over-Month Retention Rate: 78.64%**.
- **Average Month-over-Month Churn Rate: 21.36%**.
- The **best month for retention was August 2023 (95.06%)**, and the worst was September 2022 (69.02%).

### 8. Cohort Month-over-Month Retention Heatmap:
- The heatmap vividly illustrates that retention rates generally decrease over time for all cohorts, as expected.
- Earlier cohorts (e.g., 2022-09, 2022-10) show a more pronounced decline in retention over longer `month_offset` periods.
- More recent cohorts still have high retention for lower `month_offset` values, but data incompleteness (NaNs) prevents observing their long-term behavior.
- There are instances of retention rates increasing for certain cohorts at later month offsets (e.g., 2022-09 cohort from month 8 to 9), which could indicate re-engagement or specific events.

### Conclusion:
MavenFlix experiences significant early churn, especially among unpaid subscribers. While recent cohorts show promising initial retention, longer-term retention remains a challenge, with only a small percentage of customers staying beyond 5 months. Understanding the factors contributing to early churn and the differences in retention across cohorts, particularly between paid and unpaid subscribers, is crucial for improving subscriber lifetime value.